### Alcune considerazioni importanti da fare

### Alcune considerazioni importanti da fare


- Esistono due dataset, uno per il vino bianco e uno per il vino rosso, il numero di features è lo stesso mentre cambiano il numero di sample dei singoli dataset: per il vino rosso abbiamo circa 1600 righe mentre per il vino bianco abbiamo 4900 righe, cioè 3 volte tanto. Questo porta in luce un problema, cioè scegliere se fondere i due dataset in uno solo aggiungengo una feature per indicare il tipo di vino oppure sviluppare parallelamente due modelli per i singoli dataset.
  - Se scegliessimo di fondere i due dataset, dovremmo gestirne uno non bilanciato, alternativamente si può pensare di fare undersampling sul dataset dei vini bianchi portando alla creazione di un unico dataset con 3200 righe (relativamente poche) o fare oversampling sul dataset dei vini rossi avendo un dataset di approsimativamente 10000 righe.
  - Se scegliessimo di lavorare con i due dataset separati i modelli potrebbero essere più performanti, ma dovremmo gestire due modelli distinti e quindi due pipeline di lavoro distinte. Inoltre, se i due dataset sono molto simili, potremmo avere dei modelli che si sovrappongono molto e quindi non sarebbe necessario sviluppare due modelli distinti.

L'approccio che trovo più ragionevole è quello di provare entrambi i metodi e di confrontare la balanced accuracy e l'accuracy (w.r.t. dataset bilanciato e dataset sbilanciati) su modelli di classificazione quali Decision Tree, Support Vector Machine e Neural Network (senza alcun tipo di preprocessing nè scelta degli iperparametri). In questo modo potremo valutare quale approccio funziona meglio per il nostro caso specifico.


In [1]:
import pandas as pd

red_wine_data = pd.read_csv('winequality-red.csv', sep=';')
white_wine_data = pd.read_csv('winequality-white.csv', sep=';')

print(red_wine_data.shape)
print(white_wine_data.shape)

(1599, 12)
(4898, 12)


In [7]:
# Concatenate the two DataFrames
wine_type = {'red': 0, 'white': 1}
wine_data = pd.concat([red_wine_data, white_wine_data], ignore_index=True)
wine_data['type'] = [wine_type['red']] * len(red_wine_data) + [wine_type['white']] * len(white_wine_data)

wine_data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,0
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6,1
6493,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5,1
6494,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6,1
6495,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7,1


In [8]:
# Primo metodo: dataset concatenato con i 3 Decision Tree, SVC e Neural Network
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# Standardize the features

X = wine_data.drop(columns=['quality'])
y = wine_data['quality']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import balanced_accuracy_score

# Create and train the classifiers
dt_classifier = DecisionTreeClassifier(random_state=42)
dt_classifier.fit(X_train, y_train)

svc_classifier = SVC(random_state=42)
svc_classifier.fit(X_train, y_train)

mlp_classifier = MLPClassifier(random_state=42)
mlp_classifier.fit(X_train, y_train)

# Evaluate the classifiers
dt_predictions = dt_classifier.predict(X_test)
svc_predictions = svc_classifier.predict(X_test)
mlp_predictions = mlp_classifier.predict(X_test)

dt_balanced_accuracy = balanced_accuracy_score(y_test, dt_predictions)
svc_balanced_accuracy = balanced_accuracy_score(y_test, svc_predictions)
mlp_balanced_accuracy = balanced_accuracy_score(y_test, mlp_predictions)

print(f"Decision Tree Balanced Accuracy: {dt_balanced_accuracy:.4f}")
print(f"SVC Balanced Accuracy: {svc_balanced_accuracy:.4f}")
print(f"MLP Balanced Accuracy: {mlp_balanced_accuracy:.4f}")


c:\Users\micha\Desktop\Materie\Rossi-Manno\rossi-manno\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Decision Tree Balanced Accuracy: 0.3638
SVC Balanced Accuracy: 0.2258
MLP Balanced Accuracy: 0.2753


I risultati ottenuto sono abbastanza bassi, ma non è un problema, in quanto non abbiamo fatto alcun tipo di preprocessing e non abbiamo scelto gli iperparametri. Inoltre, i modelli sono stati addestrati su un dataset sbilanciato, quindi è normale che le performance siano basse. Ora verifichiamo il caso in cui alleno gl stessi modelli su i due dataset separati e vediamo se le performance migliorano.



In [11]:
# Secondo metodo: datasets separati con Decision Tree, SVC e Neural Network
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# Standardize the features

# Prima il dataset del vino rosso
X_red = red_wine_data.drop(columns=['quality'])
y_red = red_wine_data['quality']

X_train, X_test, y_train, y_test = train_test_split(X_red, y_red, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Create and train the classifiers
dt_classifier = DecisionTreeClassifier(random_state=42)
dt_classifier.fit(X_train, y_train)

svc_classifier = SVC(random_state=42)
svc_classifier.fit(X_train, y_train)

mlp_classifier = MLPClassifier(random_state=42)
mlp_classifier.fit(X_train, y_train)

# Evaluate the classifiers
dt_predictions = dt_classifier.predict(X_test)
svc_predictions = svc_classifier.predict(X_test)
mlp_predictions = mlp_classifier.predict(X_test)

dt_accuracy = accuracy_score(y_test, dt_predictions)
svc_accuracy = accuracy_score(y_test, svc_predictions)
mlp_accuracy = accuracy_score(y_test, mlp_predictions)

results_red = {
    'Decision Tree': dt_accuracy,
    'SVC': svc_accuracy,
    'MLP': mlp_accuracy
}

# Ora il dataset del vino bianco
X_white = white_wine_data.drop(columns=['quality'])
y_white = white_wine_data['quality']

X_train, X_test, y_train, y_test = train_test_split(X_white, y_white, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create and train the classifiers
dt_classifier = DecisionTreeClassifier(random_state=42)
dt_classifier.fit(X_train, y_train)

svc_classifier = SVC(random_state=42)
svc_classifier.fit(X_train, y_train)

mlp_classifier = MLPClassifier(random_state=42)
mlp_classifier.fit(X_train, y_train)

# Evaluate the classifiers
dt_predictions = dt_classifier.predict(X_test)
svc_predictions = svc_classifier.predict(X_test)
mlp_predictions = mlp_classifier.predict(X_test)

dt_accuracy = accuracy_score(y_test, dt_predictions)
svc_accuracy = accuracy_score(y_test, svc_predictions)
mlp_accuracy = accuracy_score(y_test, mlp_predictions)

results_white = {
    'Decision Tree': dt_accuracy,
    'SVC': svc_accuracy,
    'MLP': mlp_accuracy
}

# Now I made the avarege of the accuracies for each classifier
average_results = {
    'Decision Tree': (results_red['Decision Tree'] + results_white['Decision Tree']) / 2,
    'SVC': (results_red['SVC'] + results_white['SVC']) / 2,
    'MLP': (results_red['MLP'] + results_white['MLP']) / 2
}

print("Average Results:")
for classifier, accuracy in average_results.items():
    print(f"{classifier}: {accuracy:.4f}")

# Voglio stampare anche i risultati dei singoli dataset
print("\nResults for Red Wine Dataset:")
for classifier, accuracy in results_red.items():
    print(f"{classifier}: {accuracy:.4f}")

print("\nResults for White Wine Dataset:")
for classifier, accuracy in results_white.items():
    print(f"{classifier}: {accuracy:.4f}")

c:\Users\micha\Desktop\Materie\Rossi-Manno\rossi-manno\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\micha\Desktop\Materie\Rossi-Manno\rossi-manno\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Average Results:
Decision Tree: 0.5864
SVC: 0.5822
MLP: 0.5935

Results for Red Wine Dataset:
Decision Tree: 0.5625
SVC: 0.6031
MLP: 0.6156

Results for White Wine Dataset:
Decision Tree: 0.6102
SVC: 0.5612
MLP: 0.5714



### Risultati ottenuti

| Modello | Balanced Accuracy | Average Accuracy | Red Wine | White Wine |
|---------|-------------------|------------------|----------|------------|
| Decision Tree | 0.3638 | 0.5864 | 0.5625 | 0.6102 |
| SVC | 0.2258 | 0.5822 | 0.6031 | 0.5612 |
| MLP | 0.2753 | 0.5935 | 0.6156 | 0.5714 |

Senza alcun tipo di preprocessing nè di fine-tuning è chiaro che lavorare con un dataset sbilanciato produce performance molto peggiori rispetto a lavorare con i singoli dataset (red e white wine). Prima di buttare questo metodo, vorrei comunque provare a bilanciare i dataset con l'undersampling e l'oversampling. Un comportamento interessante da notare è che il Decision Tree ha una media più bassa rispetto agli altri modelli, ma in generale performa meglio sul dataset del vino bianco mentre esiste un comportamento contrario nel dataset del vino rosso, dove la SVC e la MLP performano meglio del Decision Tree. Queste informazioni potrebbero essere riutilizzati nel momento in cui dovremmo scegliere il modello giusto nel caso segliessimo di seguire il metodo di allenare modelli paralleli per i dataset separati.

Nelle successive sezioni effettuerò l'oversampling del dataset dei vini rossi e l'undersampling dei vini bianchi.
- *Oversampling*: al fine di evitare overfitting con il resampling normale in cui le righe vengono estratte e concatenate al dataset originale creando duplicati, ho deciso di utilizzare SMOTE.

In [13]:
# Proviamo a fare oversampling sul red wine portando il dataset
# ad avere lo stesso numero di righe di quello bianco utilizzando SMOTE
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='auto', random_state=42)

# Fit and apply SMOTE to the red wine dataset
X_red_resampled, y_red_resampled = smote.fit_resample(X_red, y_red)
print(f"Resampled shape: {X_red_resampled.shape}")

X_red_resampled

Resampled shape: (4086, 11)


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,7.400000,0.700000,0.000000,1.900000,0.076000,11.000000,34.000000,0.997800,3.510000,0.560000,9.400000
1,7.800000,0.880000,0.000000,2.600000,0.098000,25.000000,67.000000,0.996800,3.200000,0.680000,9.800000
2,7.800000,0.760000,0.040000,2.300000,0.092000,15.000000,54.000000,0.997000,3.260000,0.650000,9.800000
3,11.200000,0.280000,0.560000,1.900000,0.075000,17.000000,60.000000,0.998000,3.160000,0.580000,9.800000
4,7.400000,0.700000,0.000000,1.900000,0.076000,11.000000,34.000000,0.997800,3.510000,0.560000,9.400000
...,...,...,...,...,...,...,...,...,...,...,...
4081,7.460685,0.358786,0.319419,2.018466,0.074485,16.757260,25.577810,0.994567,3.253351,0.719419,11.569918
4082,8.293899,0.365820,0.393055,2.040515,0.059241,13.176834,29.000000,0.995526,3.159099,0.772154,10.996139
4083,7.729226,0.478521,0.326338,2.260916,0.075317,11.073933,19.390837,0.992978,3.213662,0.713169,12.519368
4084,8.128720,0.523680,0.157238,2.240233,0.067690,35.195346,49.333130,0.994221,3.388279,0.723564,12.565524


Ora che il dataset del vino rosso possiede un numero di righe confrontabili con quello del vino bianco, unisco il dataset resampled con quello bainco e rieffettuo il test sulla balanced accuracy dei 3 modelli.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# Standardize the features

# Ricreo prima il dataset del vino rosso con X e y resampled
red_wine_resampled = pd.concat([pd.DataFrame(X_red_resampled, columns=X_red.columns), 
                                 pd.Series(y_red_resampled, name='quality')], axis=1)

wine_data_resampled = pd.concat([red_wine_resampled, white_wine_data], ignore_index=True)
wine_data_resampled['type'] = [wine_type['red']] * len(red_wine_resampled) + [wine_type['white']] * len(white_wine_data)

print(wine_data_resampled['type'].value_counts())

wine_data_resampled

type
1    4898
0    4086
Name: count, dtype: int64


Nel nuovo dataset concatenato ci sono 4898 sample di vino bianco e 4086 sample di vino rosso, quindi direi che il dataset è abbastanza bilanciato, con un rapporto di 1.2:1 tra il vino bianco e il vino rosso.

In [30]:

X = wine_data_resampled.drop(columns=['quality'])
y = wine_data_resampled['quality']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import balanced_accuracy_score, accuracy_score

# Create and train the classifiers
dt_classifier = DecisionTreeClassifier(random_state=42)
dt_classifier.fit(X_train, y_train)

svc_classifier = SVC(random_state=42)
svc_classifier.fit(X_train, y_train)

mlp_classifier = MLPClassifier(random_state=42)
mlp_classifier.fit(X_train, y_train)

# Evaluate the classifiers
dt_predictions = dt_classifier.predict(X_test)
svc_predictions = svc_classifier.predict(X_test)
mlp_predictions = mlp_classifier.predict(X_test)

dt_balanced_accuracy = balanced_accuracy_score(y_test, dt_predictions)
svc_balanced_accuracy = balanced_accuracy_score(y_test, svc_predictions)
mlp_balanced_accuracy = balanced_accuracy_score(y_test, mlp_predictions)

dt_accuracy = accuracy_score(y_test, dt_predictions)
svc_accuracy = accuracy_score(y_test, svc_predictions)
mlp_accuracy = accuracy_score(y_test, mlp_predictions)

print(f"Decision Tree Balanced Accuracy: {dt_balanced_accuracy:.4f}")
print(f"SVC Balanced Accuracy: {svc_balanced_accuracy:.4f}")
print(f"MLP Balanced Accuracy: {mlp_balanced_accuracy:.4f}")

print(f"Decision Tree Accuracy: {dt_accuracy:.4f}")
print(f"SVC Accuracy: {svc_accuracy:.4f}")
print(f"MLP Accuracy: {mlp_accuracy:.4f}")

c:\Users\micha\Desktop\Materie\Rossi-Manno\rossi-manno\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Decision Tree Balanced Accuracy: 0.6452
SVC Balanced Accuracy: 0.5700
MLP Balanced Accuracy: 0.6009
Decision Tree Accuracy: 0.6956
SVC Accuracy: 0.6194
MLP Accuracy: 0.6450


# Risultati ottenuti su dataset artificiale bilanciato
| Modello | Balanced Accuracy - Oversampling | Accuracy - Oversampling |
|---------|-------------------|-------------------|
| Decision Tree | 0.6452 | 0.6956 |
| SVC | 0.5700 | 0.6194 |
| MLP | 0.6009 | 0.6450 |

I risultati ottenuti sono molto migliori rispetto a quelli ottenuti con il dataset sbilanciato, sia per quanto riguarda la balanced accuracy (che in questo caso è meno rilevante dal momento che il dataset è stato bilanciato) che l'accuracy. In particolare, il Decision Tree ha ottenuto la migliore performance, seguito dalla MLP e dalla SVC. Questo suggerisce che il Decision Tree potrebbe essere il modello più adatto per questo dataset bilanciato. Inoltre, questi risultati sono molto migliori di quelli ottenuti con i dataset separati, il che suggerisce che la fusione dei dataset potrebbe essere una buona strategia per migliorare le performance del modello.